# DATA PREPARATION AND ANALYZING OF ENERGY CONSUMPTION

# GROUP 14

# STUDENT ID'S: 21066354 , 21066913 , 21046749 ,  21061342




In [1]:
# IMPORTING AND START A SPARK SESSION
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('COVID-19 Data') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/03 10:50:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# READ DATA COLLECTED
d1 = (spark.read
          .format("csv")
          .options(delimiter=";", header=True)
          .load("hourly-meter-data.csv"))

In [3]:
# convert dataframe to pandas dataframe
d1.toPandas()

,Meter ID,Date,Energy
0,2200015266516,2020-03-10T18:00:00+00:00,1.454
1,2200015266516,2020-03-11T15:00:00+00:00,2.254
2,2200015266516,2020-03-12T01:00:00+00:00,0.525
3,2200015266516,2020-03-13T07:00:00+00:00,0.587
4,2200015266516,2020-03-13T08:00:00+00:00,0.594
...,...,...,...
3751305,2200042997094,2020-02-15T02:00:00+00:00,0.973
3751306,2200042997094,2020-02-15T10:00:00+00:00,0.464
3751307,2200042997094,2020-02-15T15:00:00+00:00,3.884
3751308,2200042997094,2020-02-15T17:00:00+00:00,2.433


In [7]:
#show statistics as a pandas dataframe
d1.describe().toPandas()

,summary,Meter ID,Date,Energy
0,count,3654917,3668310,3668310
1,mean,2.203377992665442E12,None,4.6128885137845295
2,stddev,3.442973160422781E10,None,19.78338179905637
3,min,2200015161200,2019-08-17T01:00:00+01:00,0.0
4,max,2600000819868,2021-08-16T01:00:00+01:00,99.99


In [8]:
# check data types
d1.dtypes

[('Meter ID', 'string'), ('Date', 'string'), ('Energy', 'string')]

In [9]:
# format data
from pyspark.sql.types import StringType,DateType,IntegerType,FloatType,DecimalType
from pyspark.sql.functions import col

d2 = d1.withColumn("Meter ID",col("Meter ID").cast(StringType())).withColumn("Date",col("Date").cast(DateType())) .withColumn("Energy",col("Energy").cast(FloatType()))
d2.dtypes

[('Meter ID', 'string'), ('Date', 'date'), ('Energy', 'float')]

In [43]:
d2.toPandas()

,Meter ID,Date,Energy
0,2200015266516,2020-03-10,1.454
1,2200015266516,2020-03-11,2.254
2,2200015266516,2020-03-12,0.525
3,2200015266516,2020-03-13,0.587
4,2200015266516,2020-03-13,0.594
...,...,...,...
3751305,2200042997094,2020-02-15,0.973
3751306,2200042997094,2020-02-15,0.464
3751307,2200042997094,2020-02-15,3.884
3751308,2200042997094,2020-02-15,2.433


In [10]:
d2.describe(['Energy']).show()

+-------+------------------+
|summary|            Energy|
+-------+------------------+
|  count|           3668310|
|   mean| 4.612888514752225|
| stddev|19.783381805842414|
|    min|               0.0|
|    max|            923.62|
+-------+------------------+



In [17]:
# check where null values exist
from pyspark.sql.functions import isnull, when, count
d2.select([count(when(isnull(c), c)).alias(c) for c in d2.columns]).show()

+--------+-----+------+
|Meter ID| Date|Energy|
+--------+-----+------+
|   96393|83000| 83000|
+--------+-----+------+



In [83]:
# remove null values
d3 = d2.dropna(how='any')

In [84]:
d3.describe().toPandas()

,summary,Meter ID,Energy
0,count,3654917,3654917
1,mean,2.203377992665442E12,4.590153622511495
2,stddev,3.442973160422781E10,19.789064265038245
3,min,2200015161200,0.0
4,max,2600000819868,923.62


In [27]:
#  creates sql table
d3.createOrReplaceTempView("meter_table")

In [29]:
# query to check how many rows exists
spark.sql("SELECT * from meter_table").show()

+-------------+----------+------+
|     Meter ID|      Date|Energy|
+-------------+----------+------+
|2200015266516|2020-03-10| 1.454|
|2200015266516|2020-03-11| 2.254|
|2200015266516|2020-03-12| 0.525|
|2200015266516|2020-03-13| 0.587|
|2200015266516|2020-03-13| 0.594|
|2200015266516|2020-03-13| 1.165|
|2200015266516|2020-03-13| 2.067|
|2200015266516|2020-03-13| 0.515|
|2200015266516|2020-03-13| 0.604|
|2200015266516|2020-03-14| 0.578|
|2200015266516|2020-03-14| 0.515|
|2200015266516|2020-03-15| 0.516|
|2200015266516|2020-03-15| 0.573|
|2200015266516|2020-03-15| 0.672|
|2200015266516|2020-03-16| 0.655|
|2200015266516|2020-03-16| 0.516|
|2200015266516|2020-03-16| 0.499|
|2200015266516|2020-03-17| 2.001|
|2200015266516|2020-03-17| 0.655|
|2200015266516|2020-03-17| 0.522|
+-------------+----------+------+
only showing top 20 rows



In [40]:
spark.sql("SELECT `Meter ID` , `Energy` FROM meter_table ORDER BY `Energy`  DESC LIMIT 20").show()

+-------------+------+
|     Meter ID|Energy|
+-------------+------+
|2200030347078|923.62|
|2200030347078|923.53|
|2200030347078| 874.2|
|2200030347078|856.35|
|2200030347078|745.61|
|2200030347078|724.75|
|2200030347078|701.67|
|2200030347078|697.11|
|2200030347078| 694.2|
|2200030347078|686.05|
|2200030347078|684.44|
|2200030347078|684.39|
|2200030347078|683.98|
|2200030347078|683.57|
|2200030347078|683.31|
|2200030347078|683.07|
|2200030347078|683.05|
|2200030347078|682.59|
|2200030347078|682.21|
|2200030347078|680.29|
+-------------+------+



In [45]:
spark.sql("SELECT `Meter ID` , `Energy`  FROM meter_table WHERE `Energy` > 0 ORDER BY `Energy`  ASC LIMIT 20").show()

+-------------+------+
|     Meter ID|Energy|
+-------------+------+
|2200017137557| 0.001|
|2200016617756| 0.001|
|2200016624433| 0.001|
|2200016617756| 0.001|
|2200016624433| 0.001|
|2200016617756| 0.001|
|2200016624433| 0.001|
|2200016617756| 0.001|
|2200016624433| 0.001|
|2200016617756| 0.001|
|2200016624433| 0.001|
|2200017137557| 0.001|
|2200016624433| 0.001|
|2200016617756| 0.001|
|2200016617756| 0.001|
|2200016617756| 0.001|
|2200016617756| 0.001|
|2200016617756| 0.001|
|2200016617756| 0.001|
|2200016617756| 0.001|
+-------------+------+



In [70]:
spark.sql("SELECT `Meter ID`, SUM(`Energy`) FROM meter_table GROUP BY `Meter ID`  ORDER BY SUM(`Energy`)  DESC LIMIT 10").show()

+-------------+------------------+
|     Meter ID|       sum(Energy)|
+-------------+------------------+
|2200030347078| 3074599.910917282|
|2200030347538|  2897821.80037117|
|2200041760597| 915564.4998267442|
|2200030363200| 750768.3800674453|
|2200030349961| 671088.3099451065|
|2200030025298| 462799.6100320816|
|2600000819868|416578.64009932615|
|2200041594742|371196.75007486343|
|2200041081741|  312806.149998188|
|2200030002389|304136.78010495193|
+-------------+------------------+



In [74]:
spark.sql("SELECT `Meter ID`, SUM(`Energy`) FROM meter_table WHERE `Energy` > 0 GROUP BY `Meter ID`  ORDER BY SUM(`Energy`)  ASC").show()

+-------------+--------------------+
|     Meter ID|         sum(Energy)|
+-------------+--------------------+
|2200017137575|0.002000000094994...|
|2200017165703|0.005000000237487...|
|2200041408589| 0.00800000037997961|
|2200041760602| 0.04399999976158142|
|2200041466711| 0.13299999863374978|
|2200017137593|  3.0619999850168824|
|2200041623793|  3.5300000309944153|
|2200016650692|  21.160000120988116|
|2200040071334|  29.644000224303454|
|2200043049270|   31.08799995633308|
|2200032203689|   62.60499972011894|
|2200017166089|    69.2290000631474|
|2200042361188|  127.09000120311975|
|2200043049304|  177.51099987328053|
|2200042486133|   206.2990011824295|
|2200042541556|  223.55300315259956|
|2200017137557|  232.11399616999552|
|2200017950043|  249.30400040815584|
|2200040471047|  252.78299974603578|
|2200017322828|   267.5740009302972|
+-------------+--------------------+
only showing top 20 rows



In [76]:
spark.sql("SELECT Year(`Date`), SUM(`Energy`) FROM meter_table GROUP BY Year(`Date`)").show()

+----------+-----------------+
|year(Date)|      sum(Energy)|
+----------+-----------------+
|      2019|3613886.664037392|
|      2020|8013483.508661219|
|      2021|5149260.334829823|
+----------+-----------------+

